In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import dask.dataframe as dd
import numpy as np
from dask.distributed import Client
import dask.multiprocessing
#


In [2]:
import umap
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px
import chart_studio.plotly as py
#

In [3]:
client = Client(n_workers=28, threads_per_worker =1, memory_limit='56GB')
#

/home/benwright/.local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42451 instead
  warnings.warn(


In [4]:
conn = 'sqlite:///scifi.db'
#

In [5]:
df = dd.read_sql_table('pages',conn,'id')
#

In [6]:
df.head(npartitions=1)

,url,content,added,updated
id,,,,
1,https://en.wikipedia.org/wiki/List_of_science_...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:54:56.615887,2024-10-12 21:54:57.030942
2,https://en.wikipedia.org/wiki/Slan,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.973417,2024-10-12 21:55:33.350622
3,https://en.wikipedia.org/wiki/The_Island_of_Do...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.985967,2024-10-12 21:55:33.440515
4,https://en.wikipedia.org/wiki/Robopocalypse,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.994391,2024-10-12 21:55:33.495511
5,https://en.wikipedia.org/wiki/John_W._Campbell...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.001689,2024-10-12 21:55:33.808913


In [7]:
len(df)

38986

In [9]:
def is_soup_novel_page(soup):
    score = 0

    should_have_strings = ('Publication date','Pages','ISBN','Author')
    s1 = 1.0/len(should_have_strings)
    for search_string in should_have_strings:
        if(soup.find(string=search_string)):
            score += s1

    is_novel_page = (score >=.75)

    return is_novel_page

def get_title(soup):
    return soup.title.text.replace(' - Wikipedia', '').replace('- Wikipedia','')

def get_novel_author(soup):

    try:
        author = soup.find(string='Author').parent.find_next_sibling('td')
        return author.text, author.get('href')
    except:
        return None, None
    
# a quick check for if a beautiful soup is for an artist article
def is_soup_author_page(soup):

    score = 0

    should_have_strings = ('Born','Genre','Occupation','Bibliography')
    s1 = 1.0/len(should_have_strings)

    for search_string in should_have_strings:
        if(soup.find(string=search_string)):
            score += s1

    is_artist_page = (score >= .75)

    return is_artist_page



# gets the type of wikipedia page. wikipedia specific, slow, and not guaranteed to be accurate, so off by default
def get_pagetype(soup = None, html = None):

    ret_val = ''

    if((html==None or html =='') and soup==None):
        return ret_val
    
    elif(soup == None):
        soup = BeautifulSoup(html,'html.parser')

    if(is_soup_novel_page(soup)):
        return 'novel'
    elif(is_soup_author_page(soup)):
        return 'author'
    
    return ret_val
#

In [10]:
def get_pagetype_for_df_wrapper(row):
    html = row['content']
    return get_pagetype(html=html)

meta_df = pd.DataFrame(columns=['pagetype'],dtype=str)
#

In [11]:
df['pagetype'] = df.apply(get_pagetype_for_df_wrapper,axis=1,meta=meta_df).compute()

2024-10-13 19:23:42,942 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,340 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,562 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,677 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,678 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,790 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,810 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-10-13 19:23:43,827 - distributed.utils_perf - WARNING - full garbage collections took

In [12]:
df2 = df[df['pagetype']!='']

In [13]:
df2.head(npartitions=5)

,url,content,added,updated,pagetype
id,,,,,
2,https://en.wikipedia.org/wiki/Slan,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.973417,2024-10-12 21:55:33.350622,novel
4,https://en.wikipedia.org/wiki/Robopocalypse,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.994391,2024-10-12 21:55:33.495511,novel
6,https://en.wikipedia.org/wiki/The_Space_Vampires,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.011547,2024-10-12 21:55:33.855602,novel
7,https://en.wikipedia.org/wiki/Twenty_Thousand_...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.018840,2024-10-12 21:55:33.951299,novel
9,https://en.wikipedia.org/wiki/Escape_on_Venus,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.034879,2024-10-12 21:55:34.050630,novel


In [14]:
def get_title_wrapper(row):
    html = row['content']
    return get_title(BeautifulSoup(html,'html.parser'))

meta_df = pd.DataFrame(columns=['title'],dtype=str)


In [15]:
df2['title'] = df2.apply(get_title_wrapper,axis=1,meta=meta_df).compute()


2024-10-13 19:29:17,927 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2024-10-13 19:29:21,339 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-10-13 19:29:21,524 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-10-13 19:29:23,625 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-10-13 19:29:23,626 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-10-13 19:29:23,679 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-10-13 19:29:23,687 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-10-13 19:29:23,700 - distributed.utils_perf - WARNING - full garbage collections took

In [16]:
df2.head(npartitions=5)

,url,content,added,updated,pagetype,title
id,,,,,,
2,https://en.wikipedia.org/wiki/Slan,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.973417,2024-10-12 21:55:33.350622,novel,Slan
4,https://en.wikipedia.org/wiki/Robopocalypse,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.994391,2024-10-12 21:55:33.495511,novel,Robopocalypse
6,https://en.wikipedia.org/wiki/The_Space_Vampires,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.011547,2024-10-12 21:55:33.855602,novel,The Space Vampires
7,https://en.wikipedia.org/wiki/Twenty_Thousand_...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.018840,2024-10-12 21:55:33.951299,novel,Twenty Thousand Leagues Under the Seas
9,https://en.wikipedia.org/wiki/Escape_on_Venus,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.034879,2024-10-12 21:55:34.050630,novel,Escape on Venus


In [17]:
novel_df = df2[df2['pagetype']=='novel']
author_df = df2[df2['pagetype']=='author']
#

In [18]:
author_df.head(npartitions=2)

,url,content,added,updated,pagetype,title
id,,,,,,
12,https://en.wikipedia.org/wiki/Dave_Wolverton,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.057727,2024-10-12 21:55:34.225879,author,Dave Wolverton
13,https://en.wikipedia.org/wiki/Jack_Vance,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.065969,2024-10-12 21:55:34.379000,author,Jack Vance
15,https://en.wikipedia.org/wiki/Alan_Dean_Foster,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.082467,2024-10-12 21:55:34.826606,author,Alan Dean Foster
24,https://en.wikipedia.org/wiki/Eric_Frank_Russell,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.173176,2024-10-12 21:55:35.610861,author,Eric Frank Russell
28,https://en.wikipedia.org/wiki/Douglas_Adams,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.204507,2024-10-12 21:55:35.980617,author,Douglas Adams


In [19]:
def get_novel_author_wrapper(row):
    html = row['content']
    return get_novel_author(BeautifulSoup(html,'html.parser'))[0]

meta_df = pd.DataFrame(columns=['novel_author'],dtype=str)


In [20]:
novel_df['novel_author'] = novel_df.apply(get_novel_author_wrapper,axis=1,meta=meta_df).compute()


2024-10-13 19:30:21,894 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2024-10-13 19:30:23,000 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-10-13 19:30:24,763 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-10-13 19:30:24,891 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-10-13 19:30:24,995 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-10-13 19:30:25,290 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-10-13 19:30:25,452 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-10-13 19:30:25,457 - distributed.utils_perf - WARNING - full garbage collections took

In [21]:
novel_df.head(npartitions=5)

,url,content,added,updated,pagetype,title,novel_author
id,,,,,,,
2,https://en.wikipedia.org/wiki/Slan,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.973417,2024-10-12 21:55:33.350622,novel,Slan,A. E. van Vogt
4,https://en.wikipedia.org/wiki/Robopocalypse,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:10.994391,2024-10-12 21:55:33.495511,novel,Robopocalypse,Daniel H. Wilson
6,https://en.wikipedia.org/wiki/The_Space_Vampires,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.011547,2024-10-12 21:55:33.855602,novel,The Space Vampires,Colin Wilson
7,https://en.wikipedia.org/wiki/Twenty_Thousand_...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.018840,2024-10-12 21:55:33.951299,novel,Twenty Thousand Leagues Under the Seas,Jules Verne
9,https://en.wikipedia.org/wiki/Escape_on_Venus,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:11.034879,2024-10-12 21:55:34.050630,novel,Escape on Venus,Edgar Rice Burroughs


In [22]:
def get_label(row):
    novel = row['title'].replace(' - Wikipedia', '').replace('- Wikipedia', '')
    author = row['novel_author']
    return f"{novel} by {author}"
meta_df = pd.DataFrame(columns=['label'],dtype=str)
novel_df['label'] = novel_df.apply(get_label,axis=1,meta=meta_df).compute()

2024-10-13 19:31:00,044 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-10-13 19:31:05,638 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)


In [23]:
def replace_special_chars(content):
    if(content == None or content == ''):
        return ''
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()
    return text.replace('\n',' ').replace('\t',' ')

In [24]:
links_df = dd.read_sql_table('links',conn,'id')


In [25]:
ps_df = df2.compute().drop(columns=['content']).merge(links_df.compute(),how='left',left_on='url',right_on='source')


2024-10-13 19:31:12,810 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2024-10-13 19:31:20,103 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-10-13 19:31:20,253 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)


In [26]:
count_df = novel_df.compute().drop(columns=['content']).merge(ps_df,how='left',left_on='url',right_on='destination')    
join_df=count_df.groupby('destination')['source'].count().reset_index()

2024-10-13 19:31:32,088 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


In [27]:
novel_df = novel_df.merge(join_df,left_on='url',right_on='destination',how='inner').compute().sort_values(by='source',ascending=False).rename(columns={'source':'rank'})

2024-10-13 19:31:39,765 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-10-13 19:31:41,467 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-10-13 19:31:45,754 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


In [28]:
novel_df.head()

,url,content,added,updated,pagetype,title,novel_author,label,destination,rank
29,https://en.wikipedia.org/wiki/Timeline_of_scie...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:26.635501,NaT,novel,Timeline of science fiction,Francis Bacon,Timeline of science fiction by Francis Bacon,https://en.wikipedia.org/wiki/Timeline_of_scie...,42
2,https://en.wikipedia.org/wiki/Ancillary_Justice,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:25.372249,NaT,novel,Ancillary Justice,Ann Leckie,Ancillary Justice by Ann Leckie,https://en.wikipedia.org/wiki/Ancillary_Justice,29
373,https://en.wikipedia.org/wiki/The_Left_Hand_of...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:22.439688,NaT,novel,The Left Hand of Darkness,Ursula K. Le Guin,The Left Hand of Darkness by Ursula K. Le Guin,https://en.wikipedia.org/wiki/The_Left_Hand_of...,29
127,https://en.wikipedia.org/wiki/Neuromancer,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:13.906189,2024-10-12 22:51:47.046269,novel,Neuromancer,William Gibson,Neuromancer by William Gibson,https://en.wikipedia.org/wiki/Neuromancer,25
341,https://en.wikipedia.org/wiki/The_Dispossessed,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 21:55:20.856686,NaT,novel,The Dispossessed,Ursula K. Le Guin,The Dispossessed by Ursula K. Le Guin,https://en.wikipedia.org/wiki/The_Dispossessed,25


In [29]:
df3 = novel_df[novel_df['novel_author']!='<NA>']

In [30]:
vectorizer = TfidfVectorizer(max_features=2000,stop_words='english',min_df =.1,max_df=.9)
meta_df = pd.Series(dtype=str)
vectors = vectorizer.fit_transform(df3['content'].map(replace_special_chars)) # might have to add .compute() after ['content'] in case dask load is lazy

In [31]:
tsne = TSNE(n_components=2, random_state=42)
reducer = umap.UMAP()
list_dense = vectors.todense().tolist()
tnse_df = pd.DataFrame(list_dense,columns = vectorizer.get_feature_names_out())
X_tsne = tsne.fit_transform(tnse_df)
X_umap = reducer.fit_transform(tnse_df)
print(tsne.kl_divergence_)


1.1879301071166992


In [107]:
N_CLUSTERS = 7

In [108]:
kmeans = KMeans(n_clusters=N_CLUSTERS,random_state=42)
kmeans.fit(tnse_df)

KMeans(n_clusters=7, random_state=42)

In [109]:
mask = df3['rank'] > 1 # df3['rank'].mean() 
mask.sum()

895

In [110]:
hover_names =df3['label'][mask] #.compute()
size = df3['rank'][mask]/df3['rank'].max()
L = len(df3[mask])

In [111]:
fig = px.scatter(x=X_tsne[mask, 0], y=-X_tsne[mask, 1],hover_name=hover_names,color=kmeans.labels_[mask], size=size)
fig.update_layout(
    title=f"t-SNE reduction of {L} TF-IDF measured Wikipedia articles on Sci-Fi Novels, weighted by number of citations",

)
fig.update(layout_coloraxis_showscale=False)
#py.plot(fig,filename=f'sci-fi-novels-htmlless-{L}',auto_open=True)
fig.show()


In [112]:
for center in kmeans.cluster_centers_:
    print(tnse_df.columns[np.argsort(center)[-15:]][::-1])

Index(['novel', 'science', 'stub', 'article', 'ender', 'collection', 'stories',
       'story', 'add', 'vte', 'expanding', 'help', 'articles', 'series',
       'novels'],
      dtype='object')
Index(['conan', 'carter', 'camp', '1978', '1967', '1968', '1972', 'swords',
       '1969', '1975', 'fantasy', '1953', '1973', 'tales', '1976'],
      dtype='object')
Index(['award', 'novel', 'best', 'le', 'guin', 'ursula', 'science', 'present',
       'bujold', 'mcmaster', 'connie', 'willis', 'robert', 'lois', '2017'],
      dtype='object')
Index(['slave', 'mars', 'rice', 'jungle', '1917', 'venus', 'edgar', 'lion',
       '1918', 'slavery', '1916', '1927', 'men', 'carter', '1914'],
      dtype='object')
Index(['novel', 'book', 'new', 'world', 'science', 'story', 'original', 'isbn',
       'archived', 'film', 'books', 'series', 'life', 'time', 'characters'],
      dtype='object')
Index(['dick', '1953', '1954', 'philip', '1955', '1964', '1963', 'report',
       '1956', 'man', 'blade', 'electric', '